In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import time
import numpy as np
import random
from model import LSTM
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
### 0.0053 loss
data = pd.read_csv("trajectory_total.csv").iloc[:, [0, 1, 2]]
data.columns=['x','y','z']
data=(data-data.mean())/data.std() #归一化  
data

torch.Size([24, 1, 3])


,x,y,z
0,-0.506642,-0.329160,0.034070
1,-0.506122,-0.329210,0.034022
2,-0.505602,-0.329259,0.033942
3,-0.505082,-0.329308,0.033831
4,-0.504562,-0.329358,0.033687
...,...,...,...
4994,-0.484190,-0.360386,0.360170
4995,-0.484148,-0.360396,0.354604
4996,-0.484106,-0.360406,0.349014
4997,-0.484063,-0.360416,0.343401


### 制作DataLoader

In [2]:
time_step=3 #时间间隔
batch_size=12 #批次数
class MyDataset(Dataset):
    def __init__(self,data,time_step):
        self.data=data
        self.time_step=time_step
    def __len__(self):
        return len(data)-self.time_step-1
    def __getitem__(self, idx):
        return np.array(data.iloc[idx:idx+self.time_step].values,dtype=np.float32),np.array(data.iloc[idx+self.time_step].values.reshape(-1,3),dtype=np.float32)
dataset=MyDataset(data,time_step=3)
train_index = list(range(int(len(data)*0.8)))
test_index = list(range(int(len(data)*0.8),int(len(data)-time_step-1)))
random.shuffle(train_index)
random.shuffle(test_index)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size,drop_last=True,sampler=train_index)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size,drop_last=True,sampler=test_index)
#输出示范
for xx,yy in train_loader:
    print(xx.shape,yy.shape)
    break

torch.Size([24, 3, 3]) torch.Size([24, 1, 3])


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def kf_predict(P0, A, Q):
    P10 = np.dot(np.dot(A, P0), A.T) + Q
    return P10


def kf_update(Xpred, P10, Z, H, R):
    V = Z - np.dot(H, Xpred)
    K = np.dot(np.dot(P10, H.T), np.linalg.pinv(np.dot(np.dot(H, P10), H.T) + R))
    X1 = Xpred + np.dot(K, V)
    P1 = np.dot(np.eye(K.shape[0]) - np.dot(K, H), P10)
    return (X1, P1, K)

nx = 1 #变量数量
# t = np.linspace(0,1,p) #时间序列
# dt = t[1] - t[0]

# 观测噪声协方差！！！！！！！！！！！！！！！！！！！！（可调整）
R = np.diag([0.1])

# # 观测值
# e = np.random.normal(0, 2, p)
# x_obs1 =test_data[:,0]+e
# x_obs2 =test_data[:,1]+e
# x_obs3 =test_data[:,2]+e

# 计算系数
A = np.array(1).reshape([nx, nx])

# 状态假设（观测）初始值
x0 = 8121.800180018
y0 = 101.79990334948
z0 = 143.252015974529
X0 = np.array(x0).reshape(nx, 1)
Y0 = np.array(y0).reshape(nx, 1)
Z0 = np.array(z0).reshape(nx, 1)

# 初始状态不确定度！！！！！！！！！！！！！！！！（可调整）
P0 = np.diag([0.1])

# 状态递推噪声协方差！！！！！！！！！！！！！！！！！！（可调整）
Q = np.diag([0.1])

###开始处理
X1_np = np.copy(X0)
X2_np = np.copy(Y0)
X3_np = np.copy(Z0)
P1_list = [P0]
P10_list = [P0]

In [4]:
hidden_size = 64
num_layers = 2
output_size = 1

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=LSTM(hidden_size, num_layers, output_size, batch_size).to(device)
loss_function = nn.MSELoss(reduce=True, size_average=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 建立优化器实例
epochs = 50
loss_list = []

c:\Users\Suchcool\.conda\envs\torch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [6]:
for i in range(epochs):
    start = time.time()
    loss_all = 0
    num = 0
    for seq_batch, label_batch in train_loader:
        seq_batch,label_batch = seq_batch.to(device),label_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(seq_batch)
        loss = 0
        for j in range(3):
            loss += loss_function(y_pred[:,:, j], label_batch[:, :, j])
        loss /= 3

        loss.backward()  # 调用loss.backward()自动生成梯度，
        optimizer.step()  # 使用optimizer.step()执行优化器，把梯度传播回每个网络
        loss_all += loss.item()
        num += 1
        # 查看模型训练的结果
    loss_all /= num
    loss_list.append(loss_all)
    print(f'epoch:{i:3} loss:{loss_all:10.8f} time:{time.time() - start:6}')

epoch:  0 loss:0.64276056 time:3.875636339187622
epoch:  1 loss:0.04276979 time:3.5156006813049316
epoch:  2 loss:0.02497329 time:3.15057635307312
epoch:  3 loss:0.02067327 time:2.991002321243286
epoch:  4 loss:0.01860567 time:3.1954550743103027
epoch:  5 loss:0.01701665 time:2.7586231231689453
epoch:  6 loss:0.01553417 time:2.7705917358398438
epoch:  7 loss:0.01404396 time:2.7586238384246826
epoch:  8 loss:0.01253357 time:2.76560640335083
epoch:  9 loss:0.01095468 time:2.773583173751831
epoch: 10 loss:0.00947018 time:2.787574529647827
epoch: 11 loss:0.00840847 time:2.7585954666137695
epoch: 12 loss:0.00780188 time:2.826443672180176
epoch: 13 loss:0.00741669 time:2.8473856449127197
epoch: 14 loss:0.00712640 time:2.8124804496765137
epoch: 15 loss:0.00689471 time:2.862346649169922
epoch: 16 loss:0.00670864 time:2.7905375957489014
epoch: 17 loss:0.00655874 time:2.809488534927368
epoch: 18 loss:0.00643651 time:2.766602039337158
epoch: 19 loss:0.00633521 time:2.7715890407562256
epoch: 20 lo

## 保存模型

In [7]:
torch.save(model.state_dict(), "./save/model_parameter.pkl")

## 模型预测

In [8]:
import torch.nn as nn
import torch
from model import LSTM
batch_size=1 #批次数
hidden_size = 64
num_layers = 2
output_size = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=LSTM(hidden_size, num_layers, output_size, batch_size).to(device) 
model.load_state_dict(torch.load("./save/model_parameter.pkl"))    # 加载模型参数  
  

<All keys matched successfully>

In [9]:
input=torch.Tensor(1,3,3).cuda()
input

tensor([[[4.7428e+30, 4.0092e-08, 3.8472e-11],
         [2.6339e+20, 1.7034e+28, 3.0478e+32],
         [1.4580e-19, 7.3313e+22, 1.8471e+25]]], device='cuda:0')

In [10]:
def model_predict(input):
    with torch.no_grad():
        return model(input)[0][0].cpu().numpy()
print(model_predict(input))

[ 0.43543684 -0.41669905  3.2352529 ]
